# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [5]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# from sklearn.model_selection import train_test_split
# from sklearn import metrics
# from sklearn.ensemble import RandomForestClassifier

# from scipy.stats import ks_2samp
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy

%matplotlib inline

In [6]:
df = pd.read_csv('previsao_de_renda.csv')
df

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,14995,2016-03-01,16006,F,False,True,0,Empresário,Secundário,Solteiro,Casa,48,13.887671,1.0,7990.58
14996,14996,2016-03-01,3722,F,False,True,0,Pensionista,Superior completo,Solteiro,Casa,57,NaN,1.0,10093.45
14997,14997,2016-03-01,6194,F,True,True,0,Assalariado,Superior completo,Casado,Casa,45,7.832877,2.0,604.82
14998,14998,2016-03-01,4922,M,True,False,0,Empresário,Superior completo,Casado,Casa,36,4.298630,2.0,3352.27


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [12]:
# 1:
y, x = patsy.dmatrices("np.log(renda) ~ data_ref + id_cliente + C(sexo) + C(posse_de_veiculo) + C(posse_de_imovel) + qtd_filhos + C(tipo_renda) + C(educacao) + C(estado_civil) + C(tipo_residencia) + idade + tempo_emprego + qt_pessoas_residencia", data = df)
x

DesignMatrix with shape (12427, 40)
  Columns:
    ['Intercept',
     'data_ref[T.2015-02-01]',
     'data_ref[T.2015-03-01]',
     'data_ref[T.2015-04-01]',
     'data_ref[T.2015-05-01]',
     'data_ref[T.2015-06-01]',
     'data_ref[T.2015-07-01]',
     'data_ref[T.2015-08-01]',
     'data_ref[T.2015-09-01]',
     'data_ref[T.2015-10-01]',
     'data_ref[T.2015-11-01]',
     'data_ref[T.2015-12-01]',
     'data_ref[T.2016-01-01]',
     'data_ref[T.2016-02-01]',
     'data_ref[T.2016-03-01]',
     'C(sexo)[T.M]',
     'C(posse_de_veiculo)[T.True]',
     'C(posse_de_imovel)[T.True]',
     'C(tipo_renda)[T.Bolsista]',
     'C(tipo_renda)[T.Empresário]',
     'C(tipo_renda)[T.Pensionista]',
     'C(tipo_renda)[T.Servidor público]',
     'C(educacao)[T.Pós graduação]',
     'C(educacao)[T.Secundário]',
     'C(educacao)[T.Superior completo]',
     'C(educacao)[T.Superior incompleto]',
     'C(estado_civil)[T.Separado]',
     'C(estado_civil)[T.Solteiro]',
     'C(estado_civil)[T.União]',


In [15]:
# 1:
df["data_ref"].value_counts()

data_ref
2015-01-01    1000
2015-02-01    1000
2015-03-01    1000
2015-04-01    1000
2015-05-01    1000
2015-06-01    1000
2015-07-01    1000
2015-08-01    1000
2015-09-01    1000
2015-10-01    1000
2015-11-01    1000
2015-12-01    1000
2016-01-01    1000
2016-02-01    1000
2016-03-01    1000
Name: count, dtype: int64

In [16]:
# 1:
df["tipo_renda"].value_counts()

tipo_renda
Assalariado         7633
Empresário          3508
Pensionista         2582
Servidor público    1268
Bolsista               9
Name: count, dtype: int64

In [17]:
# 1:
df["educacao"].value_counts()

educacao
Secundário             8895
Superior completo      5335
Superior incompleto     579
Primário                165
Pós graduação            26
Name: count, dtype: int64

In [19]:
# 1:
df["estado_civil"].value_counts()

estado_civil
Casado      10534
Solteiro     1798
União        1078
Separado      879
Viúvo         711
Name: count, dtype: int64

In [20]:
# 1:
df["tipo_residencia"].value_counts()

tipo_residencia
Casa             13532
Com os pais        675
Governamental      452
Aluguel            194
Estúdio             83
Comunitário         64
Name: count, dtype: int64

In [27]:
# 1:
reg = smf.ols("np.log(renda) ~ data_ref + id_cliente + C(sexo) + C(posse_de_veiculo) + C(posse_de_imovel) + qtd_filhos + C(tipo_renda) + C(educacao, Treatment(2)) + C(estado_civil) + C(tipo_residencia, Treatment(1)) + idade + tempo_emprego + qt_pessoas_residencia", data = df).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.359
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     178.0
Date:                Thu, 15 Feb 2024   Prob (F-statistic):               0.00
Time:                        05:13:56   Log-Likelihood:                -13551.
No. Observations:               12427   AIC:                         2.718e+04
Df Residuals:                   12387   BIC:                         2.748e+04
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================
                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------
Intercept                                             6.5441      0.220     29.697      0.000       6.112       6.976
data_ref[T.2015-02-01]                                0.0028      0.035      0.079      0.937      -0.067       0.072
data_ref[T.2015-03-01]                                0.0501      0.036      1.411      0.158      -0.020       0.120
data_ref[T.2015-04-01]                                0.0489      0.035      1.381      0.167      -0.021       0.118
data_ref[T.2015-05-01]                               -0.0186      0.035     -0.525      0.599      -0.088       0.051
data_ref[T.2015-06-01]                                0.0730      0.035      2.057      0.040       0.003       0.142
data_ref[T.2015-07-01]                                0.0281      0.035      0.795      0.427      -0.041       0.097
data_ref[T.2015-08-01]                                0.0002      0.035      0.007      0.994      -0.069       0.070
data_ref[T.2015-09-01]                               -0.0079      0.035     -0.223      0.823      -0.077       0.061
data_ref[T.2015-10-01]                                0.0235      0.035      0.664      0.507      -0.046       0.093
data_ref[T.2015-11-01]                                0.0202      0.035      0.569      0.569      -0.049       0.090
data_ref[T.2015-12-01]                                0.0727      0.036      2.041      0.041       0.003       0.142
data_ref[T.2016-01-01]                               -0.0376      0.035     -1.060      0.289      -0.107       0.032
data_ref[T.2016-02-01]                               -0.0069      0.035     -0.194      0.846      -0.076       0.063
data_ref[T.2016-03-01]                               -0.0393      0.036     -1.099      0.272      -0.109       0.031
C(sexo)[T.M]                                          0.7885      0.015     53.817      0.000       0.760       0.817
C(posse_de_veiculo)[T.True]                           0.0424      0.014      2.999      0.003       0.015       0.070
C(posse_de_imovel)[T.True]                            0.0798      0.014      5.684      0.000       0.052       0.107
C(tipo_renda)[T.Bolsista]                             0.1987      0.241      0.823      0.410      -0.274       0.672
C(tipo_renda)[T.Empresário]                           0.1564      0.015     10.474      0.000       0.127       0.186
C(tipo_renda)[T.Pensionista]                         -0.2895      0.241     -1.199      0.231      -0.763       0.184
C(tipo_renda)[T.Servidor público]                     0.0574      0.022      2.583      0.010       0.014       0.101
C(educacao, Treatment(2))[T.Primário]                 0.0200      0.072      0.277      0.782      -0.121       0.161
C(educacao, Tre

In [28]:
# 1:
print("Analisando alguns parâmetros consegui perceber que açguns fazem sentido ter mais outros podem ser retirados porque são insignificantes.")

Analisando alguns parâmetros consegui perceber que açguns fazem sentido ter mais outros podem ser retirados porque são insignificantes.


In [29]:
# 2: 
reg = smf.ols("np.log(renda) ~ id_cliente + C(sexo) + C(posse_de_veiculo) + C(posse_de_imovel) + qtd_filhos + C(estado_civil) + idade + tempo_emprego + qt_pessoas_residencia", data = df).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     550.7
Date:                Thu, 15 Feb 2024   Prob (F-statistic):               0.00
Time:                        05:21:39   Log-Likelihood:                -13664.
No. Observations:               12427   AIC:                         2.735e+04
Df Residuals:                   12414   BIC:                         2.745e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                       6.7288      0.220     30.580      0.000       6.297       7.160
C(sexo)[T.M]                    0.7718      0.015     52.656      0.000       0.743       0.801
C(posse_de_veiculo)[T.True]     0.0554      0.014      3.904      0.000       0.028       0.083
C(posse_de_imovel)[T.True]      0.0843      0.014      6.080      0.000       0.057       0.111
C(estado_civil)[T.Separado]     0.3064      0.112      2.734      0.006       0.087       0.526
C(estado_civil)[T.Solteiro]     0.2514      0.110      2.292      0.022       0.036       0.466
C(estado_civil)[T.União]       -0.0293      0.025     -1.159      0.247      -0.079       0.020
C(estado_civil)[T.Viúvo]        0.3286      0.116      2.824      0.005       0.100       0.557
id_cliente                  -2.728e-06   1.37e-06     -1.992      0.046   -5.41e-06   -4.33e-08
qtd_filhos                     -0.2429      0.109     -2.235      0.025      -0.456      -0.030
idade                           0.0045      0.001      5.769      0.000       0.003       0.006
tempo_emprego                   0.0610      0.001     59.057      0.000       0.059       0.063
qt_pessoas_residencia           0.2666      0.109      2.457      0.014       0.054       0.479
==============================================================================
Omnibus:                        1.469   Durbin-Watson:                   2.025
Prob(Omnibus):                  0.480   Jarque-Bera (JB):                1.445
Skew:                           0.025   Prob(JB):                        0.485
Kurtosis:                       3.015   Cond. No.                     4.85e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.85e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [30]:
# 2: 
print("De acordo com o R quadrado ajustado, ele diminuiu, portanto podemos dizer que piorou. ")
print("De acordo com o AIC, ele aumentou no novo modelo, portanto podemos dizer que piorou.")

De acordo com o R quadrado ajustado, ele diminuiu, portanto podemos dizer que piorou. 
De acordo com o AIC, ele aumentou no novo modelo, portanto podemos dizer que piorou.


In [32]:
# 2:
print("O parâmetro de estado civil, podemos dizer que ele foi o único que teve uma mudança significativa.")

O parâmetro de estado civil, podemos dizer que ele foi o único que teve uma mudança significativa.


In [34]:
# 3: 
reg = smf.ols("np.log(renda) ~ id_cliente + C(sexo) + C(posse_de_veiculo) + C(posse_de_imovel) + qtd_filhos + idade + tempo_emprego ", data = df).fit()
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     941.7
Date:                Thu, 15 Feb 2024   Prob (F-statistic):               0.00
Time:                        05:29:20   Log-Likelihood:                -13671.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12419   BIC:                         2.742e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                       7.2487      0.036    200.435      0.000       7.178       7.320
C(sexo)[T.M]                    0.7693      0.015     52.801      0.000       0.741       0.798
C(posse_de_veiculo)[T.True]     0.0551      0.014      3.915      0.000       0.028       0.083
C(posse_de_imovel)[T.True]      0.0840      0.014      6.067      0.000       0.057       0.111
id_cliente                  -2.808e-06   1.37e-06     -2.052      0.040   -5.49e-06   -1.25e-07
qtd_filhos                      0.0243      0.009      2.809      0.005       0.007       0.041
idade                           0.0049      0.001      6.382      0.000       0.003       0.006
tempo_emprego                   0.0609      0.001     59.004      0.000       0.059       0.063
==============================================================================
Omnibus:                        1.389   Durbin-Watson:                   2.026
Prob(Omnibus):                  0.499   Jarque-Bera (JB):                1.359
Skew:                           0.023   Prob(JB):                        0.507
Kurtosis:                       3.021   Cond. No.                     5.42e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.42e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [35]:
# 3: 
print("Continuando a fazer a remoção dos dados insignificantes, chegamos no modelo final e de acordo com o R quadrado ajustado e o AIC, ele continua sendo um modelo pior que o primeiro.")

Continuando a fazer a remoção dos dados insignificantes, chegamos no modelo final e de acordo com o R quadrado ajustado e o AIC, ele continua sendo um modelo pior que o primeiro.
